The code below selects 16 128x128 tiles for each image and mask based on the maximum number of tissue pixels. The kernel also provides computed image stats. Please check my kernels to see how to use this data. 
![](https://i.ibb.co/RzSWP56/convert.png)

# Imports

In [ ]:
import os
import cv2
import skimage.io
from tqdm.notebook import tqdm
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
from glob import glob
## Setting the seeds for Reproducibility.
seed = 3141
np.random.seed(seed)

# Loading Data

In [ ]:
df = pd.read_csv('../input/prostate-cancer-grade-assessment/train.csv')
df = df[df['data_provider'] == 'karolinska'].reset_index(drop=True)
ids = df['image_id'].values
df.head()

# Constant Varibles

In [ ]:
TRAIN = '../input/prostate-cancer-grade-assessment/train_images/'
MASKS = '../input/prostate-cancer-grade-assessment/train_label_masks/'
OUT_TRAIN = 'train.zip'
OUT_MASKS = 'masks.zip'

# Modifable Variables

In [ ]:
sz = 128
N = 16
patients = 10

# Demo of Size of Images

In [ ]:
img = skimage.io.MultiImage(os.path.join(TRAIN,ids[0]+'.tiff'))
img[0].shape, img[1].shape, img[2].shape

# Function to create tiles

In [ ]:
def tile(img, mask):
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    mask = np.pad(mask,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=0)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    mask = mask.reshape(mask.shape[0]//sz,sz,mask.shape[1]//sz,sz,3)
    mask = mask.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        mask = np.pad(mask,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=0)
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    mask = mask[idxs]
    return img, mask


# create_tiles and Labels

In [ ]:
X = []
Y = []
for name in tqdm(ids[0:patients], total=patients):
    img = skimage.io.MultiImage(os.path.join(TRAIN,name+'.tiff'))[-1]
    mask = skimage.io.MultiImage(os.path.join(MASKS,name+'_mask.tiff'))[-1]
    img, mask = tile(img,mask)
    k = 0
    while k < N:
        label = 0
        if 2 in np.unique(mask[k, ]):
            label = 1
        X.append(img[k, ])
        Y.append(label)
        k += 1

In [ ]:
(X)=np.array(X)
(Y)=np.array(Y)

In [ ]:
X.shape,Y.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X, Y,random_state=0,test_size=0.2)

# Fill Remaining Notebook.....!

In [ ]:
print('x shape:', X.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
plt.figure(figsize=(20,8))
for i in range(10,18):
    plt.subplot(231 + (i))
    plt.imshow(x_train[i][:, :, 0], cmap="pink")

In [ ]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

In [ ]:
#Conv2d data_format parameter we use 'channel_last' for imgs

def create_cnn(input_shape=(28,28,1), n_classes = 10):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid' ))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', activation='relu', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), padding='valid', strides=2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    #Optimizer
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
    #Compiling the model
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
model = create_cnn(input_shape=(28,28,1), n_classes = 10)

In [ ]:
# model.summary()

In [ ]:
#defining these prior to model to increase readability and debugging
batch_size = 64
epochs = 10

In [ ]:
# Fit the Model
history=model.fit
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size), epochs = epochs, 
                              validation_data = (x_test, y_test), verbose=1, steps_per_epoch=x_train.shape[0] // batch_size,)

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'])
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train','Test'])
plt.grid()
plt.show()

In [ ]:
def predict_single(model, img):
    return np.argmax(model.predict(img[np.newaxis, ]))

In [ ]:
img = x_test[0, ]
print (img.shape)

In [ ]:
plt.figure(figsize=(20,8))
plt.subplot(241)
plt.imshow(img[:, :, 0], cmap="gray")

In [ ]:
prediction = predict_single(model, img)
print (prediction)